In [1]:
import re
import string

import pandas as pd
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [2]:
# функции для очистки текста
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_HTML(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_numbers(text):
    num = re.compile(r'\d+')
    return num.sub(r'', text)

def remove_spaces(text):
    spaces = re.compile(r'  ')
    return spaces.sub(r' ', text)

def remove_punct(text):
    punct = re.compile(r'[^\w\s]')
    return punct.sub(r'', text) 

def remove_months(text):
    months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь',
              'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь']
    for elm in months:
        text = text.replace(elm, '')
    return text

def remove_wordsNumbers(text):
    numbers = ['год', 'млн', 'тыс', 'км', 'млрд', 'тонна', 'возраст', 'гг', 'назад']
    for i in numbers:
        text = text.replace(i, '')
    return text
    
def remove_useless(text):
    useless = ['это', 'который', 'тасс', 'twitter', 'понедельник', 'вторник', 'среда',
               'четверг', 'пятница', 'суббота', 'воскресенье']
    for _ in useless:
        text = text.replace(_, '')
    return text

def clean_text(text):
    text = remove_URL(text)
    text = remove_HTML(text)
    text = remove_numbers(text)
    text = remove_punct(text)
    return text

# функция лемматизации
mystem = Mystem()
russian_stopwords = stopwords.words("russian")

def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords \
              and token != " " \
              and token.strip() not in punctuation]

    text = " ".join(tokens)

    return text

In [3]:
df_tass = pd.read_csv("C:/Users/hp/TASS_news/nonan/TASS_texts_nonan_FINAL.csv")

In [4]:
df_tass.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Head,Lead,Text,Rubric,Classi,Clean_text
453257,453257,100005,100005,Сборные Латвии и Австрии по хоккею не будут пр...,Причины отмены занятий не называются,"СОЧИ, 8 февраля. /ИТАР-ТАСС/. Ледовые трениров...",sport,2,сочи итартасс ледовый тренировка мужской сборн...
453258,453258,100006,100006,"""Миннесота"" без Алексея Шведа проиграла клубу ...","Российский защитник попал в заявку на игру, но...","ВАШИНГТОН, 8 февраля. /ИТАР-ТАСС/. Команда ""Ми...",sport,2,вашингтон итартасс команда миннесота тимбервул...
453259,453259,100007,100007,Российские олимпийцы одолжили немецким лыжника...,Изначально просьбу немцев отклонили сборные Ав...,"БЕРЛИН, 8 февраля. /Корр. ИТАР-ТАСС Антон Долг...",sport,2,берлин корра итартасс антон долгунов российски...
453260,453260,100008,100008,Олимпийская чемпионка в сноуборде канадка Рике...,Рикер сломала запястье во время тренировки три...,"ОТТАВА, 8 февраля. /ИТАР-ТАСС/. Канадка Маэль ...",sport,2,оттава итартасс канадка маэль рикер выигрывать...
453261,453261,100009,100009,Сборная России проиграла второй матч команде А...,Вероника Кудерметова в двух сетах уступила Сам...,"СИДНЕЙ, 8 февраля. /ИТАР-ТАСС/. Сборная России...",sport,2,сидней итартасс сборная россия проигрывать ком...


In [5]:
df_tass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453262 entries, 0 to 453261
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Unnamed: 0      453262 non-null  int64 
 1   Unnamed: 0.1    453262 non-null  int64 
 2   Unnamed: 0.1.1  453262 non-null  int64 
 3   Head            453262 non-null  object
 4   Lead            453262 non-null  object
 5   Text            453262 non-null  object
 6   Rubric          453262 non-null  object
 7   Classi          453262 non-null  int64 
 8   Clean_text      453262 non-null  object
dtypes: int64(4), object(5)
memory usage: 31.1+ MB


In [6]:
# векторизация

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer()
train_tfidf_vec = vec.fit(df_tass['Clean_text'])  
vec_train = train_tfidf_vec.transform(df_tass['Clean_text'])

In [7]:
print("Shape of the train embedded vector is {}".format(vec_train.shape))

Shape of the train embedded vector is (453262, 411648)


In [8]:
x = vec_train
y = df_tass['Classi']

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [10]:
X_train.shape

(362609, 411648)

In [11]:
y_test.shape

(90653,)

In [12]:
# наивный Байес
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

mnb = MultinomialNB(alpha=1).fit(X_train, y_train)
predicted_mnb = mnb.predict(X_test)

In [13]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, mnb.predict_proba(X_test), multi_class = 'ovo')

0.9893296923892917